In [6]:
import os
import re
import json

def parse_recipe(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    lines = text.splitlines()
    title = lines[0].strip()

    # Metadata
    category_match = re.search(r"Category:\s*(.*)", text)
    category = category_match.group(1).strip() if category_match else None

    prep = re.search(r"Prep Time:\s*(.*?)\|", text)
    cook = re.search(r"Cook Time:\s*(.*?)\|", text)
    servings = re.search(r"Servings?:\s*(.*)", text)

    # Ingredients
    ingredients_section = re.search(r"Ingredients:\n(.*?)\nDirections:", text, re.S)
    ingredients = []
    if ingredients_section:
        for line in ingredients_section.group(1).splitlines():
            line = line.strip()
            if line and not line.isupper():  # skip section headers like "SLOPPY JOES"
                ingredients.append(line)

    # Directions
    directions_section = re.search(r"Directions:\n(.*?)(?:\nNotes:|\nNutrition:|\nSource:|\Z)", text, re.S)
    directions = []
    if directions_section:
        for line in directions_section.group(1).splitlines():
            if line.strip():
                directions.append(line.strip())

    # Notes
    notes_section = re.search(r"Notes:\n(.*?)(?:\nNutrition:|\nSource:|\Z)", text, re.S)
    notes = []
    if notes_section:
        notes = [n.strip() for n in notes_section.group(1).splitlines() if n.strip()]

    # Nutrition
    nutrition_section = re.search(r"Nutrition:\n(.*?)(?:\nSource:|\Z)", text, re.S)
    nutrition = {}
    if nutrition_section:
        for line in nutrition_section.group(1).splitlines():
            if ":" in line:
                k, v = line.split(":", 1)
                nutrition[k.strip()] = v.strip()

    # Source
    source_match = re.search(r"Source:\s*(.*)", text)
    source = source_match.group(1).strip() if source_match else None

    return {
        "title": title,
        "category": category,
        "prep_time": prep.group(1).strip() if prep else None,
        "cook_time": cook.group(1).strip() if cook else None,
        "servings": servings.group(1).strip() if servings else None,
        "ingredients": ingredients,
        "directions": directions,
        "notes": notes,
        "nutrition": nutrition,
        "source": source
    }

def parse_all_recipes(folder="Pantry"):
    recipe_list = []
    for filename in os.listdir(folder):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder, filename)
            recipe_list.append(parse_recipe(file_path))
    return recipe_list

if __name__ == "__main__":
    recipes = parse_all_recipes("Pantry")

    with open("recipes.json", "w", encoding="utf-8") as f:
        json.dump(recipes, f, indent=2, ensure_ascii=False)

    print(f"✅ Parsed {len(recipes)} recipes from Pantry and saved to recipes.json")


✅ Parsed 24 recipes from Pantry and saved to recipes.json
